# Reporte de promesas acumuladas en el mes

In [2]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/Reportes diarios/Promesas Mes'
mes = 20201101

In [4]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [5]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')

In [6]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

# Liverpool

In [6]:
#Hacemos la consulta referente a las gestiones de Liverpool
sql_gest_liv = cursor.callproc('liverpool_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_liv = pd.DataFrame(result.fetchall())
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']
gestion_liv['credito'] = pd.to_numeric(gestion_liv['credito'])
gestion_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios
70,9219811,12,5,13000012435603,CONCEPCION DEL CARMEN TORRES CASANOVA,8115322132,celular,2020-11-30 10:19:38,HERNANDEZMEMAGU,GUADALUPE MAYELA HERNANDEZ MENDITA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-11-30,1368.00,"8115322132 TT SE COMUNICA, NO DA MOTIVO DEL AT..."
169,9219184,12,3,1400000008415568,JOSE HERIBERTO BENJAMIN DIAZ PEREZ,2414071815,celular,2020-11-30 09:49:05,RIOSTOMACR,MARY CRUZ RIOS TOLEDO,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-11-30,1737.00,2414071815 //ESPOSA DE TT LIQUIDARA CUENTA POR...
189,9219060,12,5,13000017650685,MARIEL EUGENIA LOPEZ CORTES,5563486320,celular,2020-11-30 09:44:47,OROZCOGOBR,OROZCO GONZALEZ BRENDA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-11-30,1170.00,5563486320 SRITA REALIZA PPP POR $3170 PRIMER ...


In [7]:
#Hacemos la consulta de la asignacion
sql_asig_liv = cursor.callproc('liverpool_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_liv = pd.DataFrame(result.fetchall())
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])
asignacion_liv.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,12,0,10000001017274,MEX,MAMA720814JK8,DEPN
1,12,0,10000005443864,TAMPS,HEGD7610029Z4,DEPN
2,12,0,10000005657620,PUE,LADF4609143I1,SER4


In [8]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,fecha_promesa,monto_promesa,comentarios,estado,rfc,division
0,9219811,12,5,13000012435603,CONCEPCION DEL CARMEN TORRES CASANOVA,8115322132,celular,2020-11-30 10:19:38,HERNANDEZMEMAGU,GUADALUPE MAYELA HERNANDEZ MENDITA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-11-30,1368.0,"8115322132 TT SE COMUNICA, NO DA MOTIVO DEL AT...",NL,TOCC661001QLA,REM4
1,9219184,12,3,1400000008415568,JOSE HERIBERTO BENJAMIN DIAZ PEREZ,2414071815,celular,2020-11-30 09:49:05,RIOSTOMACR,MARY CRUZ RIOS TOLEDO,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,2020-11-30,1737.0,2414071815 //ESPOSA DE TT LIQUIDARA CUENTA POR...,TLAX,DIPH660401TI3,REM9
2,9219060,12,5,13000017650685,MARIEL EUGENIA LOPEZ CORTES,5563486320,celular,2020-11-30 09:44:47,OROZCOGOBR,OROZCO GONZALEZ BRENDA,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-11-30,1170.0,5563486320 SRITA REALIZA PPP POR $3170 PRIMER ...,DF,LOCM820525RY5,REM4


In [19]:
#Obtenemos datos de las promesas de Liverpool
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_liv = pd.pivot_table(consolidado_liv2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,Division,NumeroPromesas,SumaPromesas
0,,4,6500.00
1,JGS3,16,35188.00
2,RCC4,1,6395.00
3,REM2,253,803925.66
4,REM4,112,428649.00
5,REM9,106,152589.52
6,SB02,388,10588579.87
7,SER4,1,1000.00
8,ab02,2,5170.00
9,pren,137,502392.00


# Bradesco 

In [20]:
#Hacemos la consulta referente a las gestiones de Bradesco
sql_gest_brad = cursor.callproc('bradescard_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_brad = pd.DataFrame(result.fetchall())
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['credito'] = pd.to_numeric(gestion_brad['credito'])
gestion_brad['monto_promesa'] = pd.to_numeric(gestion_brad['monto_promesa'])
gestion_brad.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios
58,9220162,1,3,4812830042336016,RAMIREZ ROMERO JOSE,4776713714,celular,2020-11-30 10:32:38,VAZQUEZSALIBE,VAZQUEZ SANCHEZ LINNETH BERENICEÃÂ,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-11-30,1405.0,4776713714 TT SE PONE EN CONTACTO PARA INDICAR...
148,9219832,1,3,2869009122280024,HUESCA CASTRO ANA ROSA,2321099321,celular,2020-11-30 10:18:05,GOMEZCALE,GOMEZ CARRILLO LETICIA,PROMESA,ENTRADA CARTEO,PROMESA DE PAGO LIQUIDACION,ENTRADA CARTEO-PROMESA DE PAGO LIQUIDACION,2020-11-30,1798.0,2321099321 SC TT DICE QUE LE LLEGO CARTEO POR...
153,9219961,1,4,5063291140806017,MONTALVO ARREDONDO GRACIELA,4445487915,celular,2020-11-30 10:17:38,RAMIREZRELEGU,RAMIREZ REYES LESLIE GUADALUPE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-11-30,900.0,4448619175// COMENTA LA SEÃâORA QUE HOY REA...


In [12]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,24,6,26]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','estado','rfc','division']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_brad.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division
0,1,3,2869000138080177,YUCATAN,JURM8405047J801,C0 4PV
1,1,3,2869000220229211,VERACRUZ,HUQL8505057F701,C0 5PV
2,1,3,2869000320518893,PUEBLA,FEHL8809278HA01,C0 5PV


In [21]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['firma_id','unegocio_id','credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad.head(3)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios,estado,rfc,division
0,9220162,1,3,4812830042336016,RAMIREZ ROMERO JOSE,4776713714,celular,2020-11-30 10:32:38,VAZQUEZSALIBE,VAZQUEZ SANCHEZ LINNETH BERENICEÃÂ,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-11-30,1405.0,4776713714 TT SE PONE EN CONTACTO PARA INDICAR...,PUEBLA,RARJ5711121L101,C0 4PV
1,9219832,1,3,2869009122280024,HUESCA CASTRO ANA ROSA,2321099321,celular,2020-11-30 10:18:05,GOMEZCALE,GOMEZ CARRILLO LETICIA,PROMESA,ENTRADA CARTEO,PROMESA DE PAGO LIQUIDACION,ENTRADA CARTEO-PROMESA DE PAGO LIQUIDACION,2020-11-30,1798.0,2321099321 SC TT DICE QUE LE LLEGO CARTEO POR...,PUEBLA,HUCA90042341201,C0 4PV
2,9219961,1,4,5063291140806017,MONTALVO ARREDONDO GRACIELA,4445487915,celular,2020-11-30 10:17:38,RAMIREZRELEGU,RAMIREZ REYES LESLIE GUADALUPE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-11-30,900.0,4448619175// COMENTA LA SEÃâORA QUE HOY REA...,SAN LUIS POTOSI,MOAG970618JG001,C1


In [25]:
#Obtenemos datos de las promesas de Bradesco
consolidado_brad2 = consolidado_brad.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_brad = pd.pivot_table(consolidado_brad2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad.fillna(0,inplace=True)
#pivot_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SumaPromesas
0,A,51,102973.72
1,B,18,28654.00
2,C0 4PV,465,908097.16
3,C0 5PV,397,758583.99
4,C0 6PV,495,20835316.98
5,C1,976,11683910.64
6,C2,298,487423.00
7,C3,368,482607.50
8,C4,174,158559.00
9,All,3242,35446125.99


# Credifiel

In [7]:
#Hacemos la consulta referente a las gestiones de Credifiel
sql_gest_cred = cursor.callproc('credifiel_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
    gestion_cred = pd.DataFrame(result.fetchall())
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen
24,8897094,1,2020-11-02,08:24:58,CR-287300,MORALES ZARATE PATRICIA,9321097494,QUINTOSARLD,QUINTOS ARVIZU LUIS DANIEL,LLAMADA A TITULAR,SEGUIMIENTO A PROMESA,,NaN,\t9321097494 SEGUIMIENTO POR WHATS,PDP,Promesa de Pago,CONTACTO
106,8897909,1,2020-11-02,09:45:05,CR-441366,SOTO LIMAS JESUS MANUEL,8681360360,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,SEGUIMIENTO A PROMESA,,NaN,8681360360 no contesta se manda mensaje vÃ­a ...,PDP,Promesa de Pago,CONTACTO
112,8897999,1,2020-11-02,09:48:21,CR-540140,ALVAREZ MUÑOZ ANGELICA,3319616535,HERNANDEZGARO,HERNANDEZ GALINDO RODRIGO,LLAMADA A TITULAR,SEGUIMIENTO A PROMESA,,NaN,3319616535 se manda mensaje via whatsapp pidie...,PDP,Promesa de Pago,CONTACTO


In [8]:
#Hacemos la consulta de la asignacion
sql_asig_cred = cursor.callproc('credifiel_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_cred = pd.DataFrame(result.fetchall())
asignacion_cred = asignacion_cred.iloc[:,[1,3,32]]
asignacion_cred.columns = ['unegocio_id','credito','division']
#asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_cred.head(3)

,unegocio_id,credito,division
0,1,CR-101922,COBRANZA EXTRAJUDICIAL
1,1,CR-101949,LIQUIDAR
2,1,CR-101952,LIQUIDAR


In [11]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',on=['unegocio_id','credito'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>=100]
consolidado_cred.head(3)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen,division
6,8903928,1,2020-11-02,16:50:16,CR-458939,CORTEZ CASTAÑEDA ELVIRA,3111064432,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-11-06,16333.00,3111064432 PLAN DE PAGOS TT PONE AL CORRIENTE ...,PDP,Promesa de Pago,CONTACTO,NaN
33,8909981,1,2020-11-03,10:50:16,CR-333406,TINOCO ELIZALDE JUAN,9611158039,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-11-15,1356.00,9611158039 TT REALIZARA PAGO 9 POR $1356 EL DI...,PDP,Promesa de Pago,CONTACTO,
47,8911773,1,2020-11-03,12:26:20,CR-464179,PACHECO VILLALOBOS JOSE FRANCISCO,6623981460,GONMORALESBM,GONZALEZ MORALES BRENDA MARLEN,LLAMADA A TITULAR,PROMESA DE PAGO PARCIAL,2020-11-30,3064.74,6623981460 TT REALIZARA PAGO 2 $3064.74 Y LOS ...,PDP,Promesa de Pago,CONTACTO,COBRANZA EXTRAJUDICIAL


In [13]:
#Obtenemos datos de las promesas de Credifiel
#consolidado_cred2 = consolidado_cred.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_cred = pd.pivot_table(consolidado_cred,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,3,2]]
pivot_cred.columns = ['Accion_Credifiel','NumeroPromesas','SumaPromesas']
pivot_cred.fillna(0,inplace=True)
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Accion_Credifiel,NumeroPromesas,SumaPromesas
0,,16,68818.51
1,COBRANZA EXTRAJUDICIAL,82,248272.63
2,EXTRAJUDICIAL,15,47549.22
3,LIQUIDAR,2,15666.60


# Banco Azteca

In [ ]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
sql_gest_baz = cursor.callproc('baz_rpt_gestiones_detallado', [mes,today])
for result in cursor.stored_results():
     gestion_baz = pd.DataFrame(result.fetchall())
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz.head(3)

In [ ]:
sample_cols_to_keep =['#FIIDCAMPANA', 'FIPAIS', 'FICANAL','FISUCURSAL','FIFOLIO','FISEMATRASMAX','FNSALDO','FNSALDOCAPITAL','FNPAGOREQ','FCTEL1','FCTIPO1','FNDIA_PAGO','FIDIASATRASOMAX','FNCAPPAGODISP','FNABONOSEMANAL','FNCAPACIDADPAGO','FILCRACTIVA','FDFECPROXPAG','FNTASAINT','CP','fcnombreos','fcappaternoos','FITERRITORIO','FCDESCTERRITORIO','FIZONA','FCDESCZONA','FIREGION','FCDESCREGION','FIGERENCIA','FCDESCGERENCIA','FCBESTTIMETOCALL','FECHA']
df_iter = pd.read_csv('/home/estadistico/Documents/Erick/Banco Azteca/Asignacion_csv/Asignacion banco azteca '+mes+'.csv', chunksize=20000, usecols=sample_cols_to_keep) 
df_lst = [] 
for df_ in df_iter: 
            tmp_df = (df_.rename(columns={col: col.lower() for col in df_.columns}))
            df_lst += [tmp_df.copy()] 
asignacion_baz = pd.concat(df_lst)